In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [1]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)
import pandas as pd
import os

C:\Users\pcrrt\Anaconda3\envs\anlp\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pcrrt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Clean Data

In [2]:
inpath2= os.path.dirname(os.path.dirname(os.getcwd()))+"\\Data\\Intermediate Data\\"
df=pd.read_csv(inpath2+'final_all_files.csv')

In [3]:
df['title2']=df['date']+" "+df['title']+" - "+df['event_type']
df=df[df['candidate']=='trump']
df2=df[['title2', 'text']]
df2.to_csv(inpath2+'trump_for_lda.tsv', sep='\t', index=False, header=False, encoding='utf8')

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [4]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

In [5]:
stop_words = {k:1 for k in stopwords.words('english')}
stop_words.update(read_stopwords("C:/Users/pcrrt/anlp21/data/jockers.stopwords"))
stop_words["'s"]=1
stop_words=list(stop_words.keys())

In [6]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [7]:
def read_docs(plotFile, stopwords):
    docs=[]
    names=[]
    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            #print(line)
            #print(line)
            cols=line.rstrip().split("\t")
            title=cols[0]
            text=cols[1]
            tokens=nltk.word_tokenize(text.lower())
            tokens=[x for x in tokens if filter(x, stopwords)]
            docs.append(tokens)
            name=title
            if name=='':
                print(text)
            names.append(name)
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [8]:
plotFile=inpath2+'/trump_for_lda.tsv'
data, doc_names=read_docs(plotFile, stop_words)

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [9]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents
dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [10]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

In [11]:
num_topics=20

Now let's run a topic model on this data using gensim's built-in LDA.

In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [13]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

topic 0:	russia germany violence report vaccine jail labor u.s. ballots protecting
topic 1:	texas russia minnesota okay farmers kurds secretary whistleblower soldiers beat
topic 2:	uh police kenosha local community feel judges safety um air
topic 3:	louisiana edwards bel kansas insurance saturday whistleblower oil approved death
topic 4:	syria turkey soldiers nato russia european italy union virus involved
topic 5:	boy votes ballots counting legal polls count momentum wisconsin company
topic 6:	corruption biden pennsylvania impeachment ukraine hunt witch corrupt press conversation
topic 7:	veterans ice mr. voted steel ohio senator aliens farmers pelosi
topic 8:	judge notre dame congratulations coney nomination professor school u.s. circuit
topic 9:	immigration okay human peace violence bless reform school interests regime
topic 10:	gon okay michigan hampshire pennsylvania tariffs constitution independence ice embassy
topic 11:	virus governors uh medical testing test okay masks business

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [14]:
topic_model=lda_model 

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s %s" % (i,v,doc_names[k], k))
    print()
    
    

russia germany violence report vaccine jail labor u.s. ballots protecting

0	0.999	09/07/2020 Labor Day Press Conference - speech 218
0	0.995	01/13/2021 Statement about the Violence at the Capitol - speech 224

texas russia minnesota okay farmers kurds secretary whistleblower soldiers beat

1	1.000	10/18/2019 FULL TRUMP RALLY: President Trump holds campaign rally in Dallas, Texas - rally 166
1	0.999	10/17/2019 Speech: Donald Trump Holds a Political Rally in Dallas, Texas - rally 29
1	0.999	10/22/2019 Donald Trump Cabinet Meeting Transcript Trump Calls Emoluments Clause of the Constitution Phony - speech 111
1	0.999	02/06/2020 News Conference Calls Russia Investigation Bullshit Talks Impeachment Acquittal - speech 120
1	0.999	02/20/2020 Donald Trump Speech Transcript at the Hope for Prisoners Graduation Ceremony - speech 126

uh police kenosha local community feel judges safety um air

2	0.999	09/01/2020 Jacob Blake protests: Trump announces funding for Kenosha police, businesses during